In [3]:
import pandas as pd # type: ignore
import datetime
import calendar
import numpy as np # type: ignore
import re
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
from sklearn.ensemble import IsolationForest

In [4]:
super_sheet = pd.ExcelFile(r"/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/Sumber_Data_Lama/SSKI/SSKI EKSTERNAL_25 Okt 2024.xlsx")

In [5]:
def prepare_dataframe(sheet_name, header_count):
    if sheet_name in ['11','16']:
        input_df = super_sheet.parse(sheet_name, header=[3, 2 + header_count])
    else:
        input_df = super_sheet.parse(sheet_name, header=[4, 3 + header_count])

    # Clean up and modify the column headers
    input_df.columns = input_df.columns.map(lambda x: str(x[0]).upper() if 'Unnamed' in str(x) else x)

    # Strip spaces in tuple columns
    input_df.columns = input_df.columns.map(lambda x: (x[0], x[1].strip()) if isinstance(x, tuple) and isinstance(x[1], str) else x)

    # Combine the multi-level column headers into a single string
    input_df.columns = input_df.columns.map(lambda x: f'{x[0]}-{x[1].capitalize()}' if isinstance(x, tuple) else x)

    first_column = input_df.iloc[:, 0]
    second_column = input_df.iloc[:, 1]

    # Filter based on the year (assuming year_to_check is a variable in scope)
    input_df = input_df.filter(regex=str(20))

    # Insert the 'NO' and 'Komponen' columns
    input_df.insert(0, 'NO', first_column)
    input_df.insert(1, 'Komponen', second_column)

    # Remove rows after the 'Keterangan' row (if it exists)
    index_keterangan = input_df[input_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index

    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        input_df = input_df.iloc[:first_keterangan_index]

    return input_df

In [6]:
def prepare_dataframe5(sheet_name):
    if sheet_name == '5a':
        header_array = [3, 4, 5]
    else:
        header_array = [2, 3, 4, 5]

    input_df = super_sheet.parse(sheet_name, header=header_array)

    input_df.columns = input_df.columns.map(lambda x: str(x[0]).upper() if 'Unnamed' in str(x) else x)

    input_df.columns = input_df.columns.map(
        lambda x: tuple(
            str(x[i]).upper() if i == 0 and 'Unnamed' in str(x[1]) else x[i].strip() if isinstance(x[i], str) else x[i]
            for i in range(len(x))
        ) if isinstance(x, tuple) else x
    )

    # Remove row with "keterangan" and rows following
    index_keterangan = input_df[input_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index

    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        input_df = input_df.iloc[:first_keterangan_index]
    
    # Gabungkan elemen kolom multi-level menjadi satu string secara fleksibel berdasarkan panjang tuple
    input_df.columns = input_df.columns.map(lambda x: '-'.join([str(i).capitalize() if isinstance(i, str) else str(i) for i in x]) if isinstance(x, tuple) else x)

    first_column = input_df.iloc[:, 0]
    second_column = input_df.iloc[:, 1]

    # Filter columns where the name contains a year (assuming year_to_check is a variable in scope)
    input_df = input_df.filter(regex=str(20))

    # Insert the 'NO' and 'Komponen' columns
    input_df.insert(0, 'NO', first_column)
    input_df.insert(1, 'Komponen', second_column)

    # Remove rows after the 'Keterangan' row (if it exists)
    index_keterangan = input_df[input_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index

    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        input_df = input_df.iloc[:first_keterangan_index]

    # Drop columns that contain only a year as their name (e.g., '2020', '2021')
    input_df = input_df.loc[:, ~input_df.columns.str.match(r'^\d{4}$')]

    return input_df

In [7]:
def split5a(df):
    claims_df = df.filter(like='Claims')
    liabilities_df = df.filter(like='Liabilities')
    common_columns = df.filter(regex='NO|Komponen')
    # Menggabungkan kolom Nomor dan Komponen ke DataFrame Claims dan Liabilities
    claims_df = pd.concat([common_columns, claims_df], axis=1)
    liabilities_df = pd.concat([common_columns, liabilities_df], axis=1)
    return claims_df, liabilities_df

def split5b(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...)
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_ld = pd.concat([common_columns, df.filter(like='Claims-Loan & deposits')], axis=1)
    liabilities_df_ld = pd.concat([common_columns, df.filter(like='Liabilities-Loan & deposits')], axis=1)
    claims_df_ds = pd.concat([common_columns, df.filter(like='Claims-Debt securities')], axis=1)
    liabilities_df_ds = pd.concat([common_columns, df.filter(like='Liabilities-Debt securities')], axis=1)
    claims_df_oi = pd.concat([common_columns, df.filter(like='Claims-Other instruments')], axis=1)
    liabilities_df_oi = pd.concat([common_columns, df.filter(like='Liabilities-Other instruments')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_ld, liabilities_df_ld, claims_df_ds, liabilities_df_ds, claims_df_oi, liabilities_df_oi

def split5d(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...) and concatenate with common columns
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_b = pd.concat([common_columns, df.filter(like='Claims-Banks')], axis=1)
    liabilities_df_b = pd.concat([common_columns, df.filter(like='Liabilities-Banks')], axis=1)
    claims_df_nb = pd.concat([common_columns, df.filter(like='Claims-Nonbanks')], axis=1)
    liabilities_df_nb = pd.concat([common_columns, df.filter(like='Liabilities-Nonbanks')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_b, liabilities_df_b, claims_df_nb, liabilities_df_nb

def split5d1(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...) and concatenate with common columns
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_r = pd.concat([common_columns, df.filter(like='Claims-Related offices')], axis=1)
    liabilities_df_r = pd.concat([common_columns, df.filter(like='Liabilities-Related offices')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_r, liabilities_df_r

def split5d2(df):
    # Extract common columns (No and Komponen)
    common_columns = df.filter(regex='NO|Komponen')
    
    # Filter for Claims and Liabilities by categories using df.filter(like=...) and concatenate with common columns
    claims_df_total = pd.concat([common_columns, df.filter(like='Claims-Total')], axis=1)
    liabilities_df_total = pd.concat([common_columns, df.filter(like='Liabilities-Total')], axis=1)
    claims_df_nb = pd.concat([common_columns, df.filter(like='Claims-Non-bank financial ins.')], axis=1)
    liabilities_df_nb = pd.concat([common_columns, df.filter(like='Liabilities-Non-bank financial ins.')], axis=1)
    claims_df_nf = pd.concat([common_columns, df.filter(like='Claims-Non-financial corporations')], axis=1)
    liabilities_df_nf = pd.concat([common_columns, df.filter(like='Liabilities-Non-financial corporations')], axis=1)
    
    # Return all the dataframes
    return claims_df_total, liabilities_df_total, claims_df_nb, liabilities_df_nb, claims_df_nf, liabilities_df_nf

In [8]:
def prepare_all(name, input_df):

    # Step 2: Filter the DataFrame based on the selected columns
    filtered_df = input_df

    # Step 3: Replace '-' with NaN and drop rows with NaN
    filtered_df.replace('-', np.nan, inplace=True)

    # Step 4: Transpose the table, so NO and Komponen become the headers
    transposed_df = filtered_df.set_index(['NO', 'Komponen']).T
    
    return transposed_df

In [27]:
import numpy as np
import pandas as pd

# Month mapping dictionary
month_mapping = {
    'Jan': 'Jan',
    'Feb': 'Feb',
    'Mar': 'Mar',
    'Apr': 'Apr',
    'Mei': 'May',
    'Jun': 'Jun',
    'Jul': 'Jul',
    'Agt': 'Aug',
    'Sep': 'Sep',
    'Okt': 'Oct',
    'Nov': 'Nov',
    'Des': 'Dec'
}

# Function to reformat Periode
def reformat_periode(periode):
    tokens = periode.split('-')
    if len(tokens) > 1:
        year = tokens[0]
        month = tokens[1]
        # print(month)
        
        # Clean the month in case there are additional characters like 'Dec*' or 'OctX'
        month = re.sub(r'[^A-Za-z]', '', month)  # Remove any non-alphabet characters
        
        return f"{year}-{month_mapping.get(month, month)}"
    return periode


def detect_outliers_iqr(name,df):
    # Pastikan DataFrame tidak kosong sebelum melanjutkan
    if df.empty:
        print("DataFrame is empty.")
        return pd.DataFrame(columns=['Komponen','Detail', 'Tahun', 'Nilai', 'Batas Bawah', 'Batas Atas', 'Outlier', 'Gap', 'last_period'])
    
    # Replace string values (including '-') with NaN
    df = df.replace('-', np.nan).applymap(lambda x: np.nan if isinstance(x, str) else x)
    
    # Get the last period for each component (assuming index represents period/year)
    last_period = df.index[-1] if len(df.index) > 0 else None

    outliers_data = []
    for column in df.columns:
        # Ensure all values in the column are numeric
        df[column] = pd.to_numeric(df[column], errors='coerce')
        col_data = df[column].dropna()
        
        if col_data.empty:
            continue  # Skip columns with no numeric data
        
        # Calculate IQR, lower bound, and upper bound
        Q1 = col_data.quantile(0.25)
        Q3 = col_data.quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Process each value and identify if it's an outlier
        for index, value in df[column].items():
            is_outlier = value < lower_bound or value > upper_bound
            
            # Calculate gap only for outliers, leave empty for non-outliers
            gap = None
            if is_outlier:
                gap = (value - upper_bound) if value > upper_bound else (lower_bound - value)
            token = index.split("-",2)
            if len(token) == 3:
                detail = token[2]
            else:
                detail = '-'


            outliers_data.append({
                'Tabel' : name,
                'No_Komponen':int(column[0]),
                'Komponen': column[1],
                'Detail' : detail,
                'Periode': index,
                'Nilai': value,
                'Batas Bawah': lower_bound,
                'Batas Atas': upper_bound,
                'Outlier': 'Ya' if is_outlier else 'Tidak',
                'Gap': gap,
                'last_period': last_period
            })
    
    # Create the DataFrame with the collected outlier data
    df_outliers = pd.DataFrame(outliers_data)

    # # Apply the function to the Periode column
    df_outliers['Periode'] = df_outliers['Periode'].apply(reformat_periode)

    
    return df_outliers

In [10]:
# name = "1"
# raw_df = prepare_dataframe(name, 2)
# checked_df = prepare_all(name, raw_df)
# checked_df = checked_df.iloc[1:]
# outliers_df = detect_outliers_iqr(name,checked_df)
# # Add a new column 'No.' that contains the index values
# outliers_df.insert(0, 'No.', range(1, len(outliers_df) + 1))
# # Display the updated DataFrame
# display(outliers_df)

# output_path = r"/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/application/data/data_pencilan_sski.csv"
# outliers_df.to_csv(output_path, index=False)

In [28]:
import pandas as pd

sheet_names = ["1","2","3","4","5a","5b","5c","5d","5d.1","5.d.2","6","7","8","9","10","11","11a","12","13","14","15","16","16a","17","18","19","20"]  

all_outliers = []

# Loop to process all sheets
for name in sheet_names:
    print(f"Processing SSKI {name}...")

    if name == "5a":
        raw_df = prepare_dataframe5(name)
        
        claims_df, liabilities_df = split5a(raw_df)
        
        claims_df = prepare_all(name, claims_df)
        liabilities_df = prepare_all(name, liabilities_df)
        
        claims_outliers = detect_outliers_iqr(name, claims_df)
        all_outliers.append(claims_outliers)

        liabilities_outliers = detect_outliers_iqr(name, liabilities_df)
        all_outliers.append(liabilities_outliers)

    elif name in ["5b", "5c"]:
        raw_df = prepare_dataframe5(name)
        
        splits = split5b(raw_df)
        
        for claims_df, liabilities_df in zip(splits[::2], splits[1::2]):
            claims_df = prepare_all(name, claims_df)
            claims_outliers = detect_outliers_iqr(name, claims_df)
            all_outliers.append(claims_outliers)

            liabilities_df = prepare_all(name, liabilities_df)
            liabilities_outliers = detect_outliers_iqr(name, liabilities_df)
            all_outliers.append(liabilities_outliers)

    elif name == "5d":
        raw_df = prepare_dataframe5(name)
        
        splits = split5d(raw_df)
        
        for claims_df, liabilities_df in zip(splits[::2], splits[1::2]):
            claims_df = prepare_all(name, claims_df)
            claims_outliers = detect_outliers_iqr(name, claims_df)
            all_outliers.append(claims_outliers)

            liabilities_df = prepare_all(name, liabilities_df)
            liabilities_outliers = detect_outliers_iqr(name, liabilities_df)
            all_outliers.append(liabilities_outliers)

    elif name == "5d.1":
        raw_df = prepare_dataframe5(name)
        
        claims_df_total, liabilities_df_total, claims_df_r, liabilities_df_r = split5d1(raw_df)
        
        for df in [claims_df_total, liabilities_df_total, claims_df_r, liabilities_df_r]:
            df = prepare_all(name, df)
            outliers = detect_outliers_iqr(name, df)
            all_outliers.append(outliers)

    elif name == "5.d.2":
        raw_df = prepare_dataframe5(name)
        
        claims_df_total, liabilities_df_total, claims_df_nb, liabilities_df_nb, claims_df_nf, liabilities_df_nf = split5d2(raw_df)
        
        for df in [claims_df_total, liabilities_df_total, claims_df_nb, liabilities_df_nb, claims_df_nf, liabilities_df_nf]:
            df = prepare_all(name, df)
            outliers = detect_outliers_iqr(name, df)
            all_outliers.append(outliers)

    else:
        raw_df = prepare_dataframe(name, 2)
        checked_df = prepare_all(name, raw_df)
        checked_df = checked_df.iloc[1:]
        outliers_df = detect_outliers_iqr(name, checked_df)
        all_outliers.append(outliers_df)

# Concatenate all outliers and export to a single CSV file
combined_outliers = pd.concat(all_outliers, ignore_index=True)
combined_outliers.insert(0, 'No.', range(1, len(combined_outliers) + 1))


combined_outliers.to_csv('/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/data/data_pencilan_sski.csv', index=False)

Processing SSKI 1...
Processing SSKI 2...


/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/253596289.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_df.replace('-', np.nan, inplace=True)
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/1490778647.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.replace('-', np.nan).applymap(lambda x: np.nan if isinstance(x, str) else x)
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/253596289.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

Processing SSKI 3...


/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/253596289.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_df.replace('-', np.nan, inplace=True)
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/1490778647.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.replace('-', np.nan).applymap(lambda x: np.nan if isinstance(x, str) else x)


Processing SSKI 4...


...

/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/253596289.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_df.replace('-', np.nan, inplace=True)
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/1490778647.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.replace('-', np.nan).applymap(lambda x: np.nan if isinstance(x, str) else x)


Processing SSKI 17...
Processing SSKI 18...


/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/253596289.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_df.replace('-', np.nan, inplace=True)
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/1490778647.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.replace('-', np.nan).applymap(lambda x: np.nan if isinstance(x, str) else x)
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/253596289.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

Processing SSKI 19...
Processing SSKI 20...


/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/253596289.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_df.replace('-', np.nan, inplace=True)
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/1490778647.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.replace('-', np.nan).applymap(lambda x: np.nan if isinstance(x, str) else x)
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_4295/253596289.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

In [29]:
df_5d = combined_outliers[combined_outliers['Tabel']=="5b"]

df_5d

,No.,Tabel,No_Komponen,Komponen,Detail,Periode,Nilai,Batas Bawah,Batas Atas,Outlier,Gap,last_period
28608,28609,5b,1,Seluruh Negara,Claims-Total,2021-Jan,618232.564,618471.779375,757793.998375,Ya,239.215375,2024-Aug*-Claims-Total
28609,28610,5b,1,Seluruh Negara,Claims-Total,2021-Feb,620782.601,618471.779375,757793.998375,Tidak,NaN,2024-Aug*-Claims-Total
28610,28611,5b,1,Seluruh Negara,Claims-Total,2021-Mar,633599.116,618471.779375,757793.998375,Tidak,NaN,2024-Aug*-Claims-Total
28611,28612,5b,1,Seluruh Negara,Claims-Total,2021-Apr,617088.816,618471.779375,757793.998375,Ya,1382.963375,2024-Aug*-Claims-Total
28612,28613,5b,1,Seluruh Negara,Claims-Total,2021-May,637820.268,618471.779375,757793.998375,Tidak,NaN,2024-Aug*-Claims-Total
...,...,...,...,...,...,...,...,...,...,...,...,...
33531,33532,5b,14,Lokasi Tidak Teridentifikasi,Liabilities-Other instruments,2024-Apr,0.000,-206.825250,344.708750,Tidak,NaN,2024-Aug*-Liabilities-Other instruments
33532,33533,5b,14,Lokasi Tidak Teridentifikasi,Liabilities-Other instruments,2024-May,0.000,-206.825250,344.708750,Tidak,NaN,2024-Aug*-Liabilities-Other instruments
33533,33534,5b,14,Lokasi Tidak Teridentifikasi,Liabilities-Other instruments,2024-Jun,0.000,-206.825250,344.708750,Tidak,NaN,2024-Aug*-Liabilities-Other instruments
33534,33535,5b,14,Lokasi Tidak Teridentifikasi,Liabilities-Other instruments,2024-Jul,0.000,-206.825250,344.708750,Tidak,NaN,2024-Aug*-Liabilities-Other instruments


In [30]:
combined_outliers.head(10)


,No.,Tabel,No_Komponen,Komponen,Detail,Periode,Nilai,Batas Bawah,Batas Atas,Outlier,Gap,last_period
0,1,1,2,Pertumbuhan Ekonomi (PDB),-,2017,5.069786,4.783962,5.408489,Tidak,NaN,2024-Sep
1,2,1,2,Pertumbuhan Ekonomi (PDB),-,2018,5.174292,4.783962,5.408489,Tidak,NaN,2024-Sep
2,3,1,2,Pertumbuhan Ekonomi (PDB),-,2019,5.018160,4.783962,5.408489,Tidak,NaN,2024-Sep
3,4,1,2,Pertumbuhan Ekonomi (PDB),-,2020,-2.065512,4.783962,5.408489,Ya,6.849474,2024-Sep
4,5,1,2,Pertumbuhan Ekonomi (PDB),-,2021,3.703055,4.783962,5.408489,Ya,1.080907,2024-Sep
5,6,1,2,Pertumbuhan Ekonomi (PDB),-,2022,5.308595,4.783962,5.408489,Tidak,NaN,2024-Sep
6,7,1,2,Pertumbuhan Ekonomi (PDB),-,2022-Jan,NaN,4.783962,5.408489,Tidak,NaN,2024-Sep
7,8,1,2,Pertumbuhan Ekonomi (PDB),-,2022-Feb,NaN,4.783962,5.408489,Tidak,NaN,2024-Sep
8,9,1,2,Pertumbuhan Ekonomi (PDB),-,2022-Mar,5.024770,4.783962,5.408489,Tidak,NaN,2024-Sep
9,10,1,2,Pertumbuhan Ekonomi (PDB),-,2022-Apr,NaN,4.783962,5.408489,Tidak,NaN,2024-Sep
